<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/Interactive_Psychrometric_Chart_with_Plotly_and_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import io # To simulate CSV file reading from a string

# 1. 기본 상수 정의 (Constants)
C_PA = 1.006  # 건공기의 정압비열 (Specific heat of dry air) (kJ/kg°C)
C_PW = 1.86   # 수증기의 정압비열 (Specific heat of water vapor) (kJ/kg°C)
H_FG0 = 2501  # 0°C에서의 물의 증발잠열 (Latent heat of vaporization at 0°C) (kJ/kg)
P_ATM = 101.325 # 표준 대기압 (Standard atmospheric pressure) (kPa)

# 2. 습공기 상태량 계산 함수 (Psychrometric Calculation Functions)
def get_saturation_vapor_pressure_kPa(temp_c):
    if temp_c < -60: return 0.0
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

def get_actual_vapor_pressure_kPa(temp_c, rh_percent):
    p_ws = get_saturation_vapor_pressure_kPa(temp_c)
    return (rh_percent / 100.0) * p_ws

def get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    p_w = get_actual_vapor_pressure_kPa(temp_c, rh_percent)
    denominator = p_atm_kPa - p_w
    if denominator <= 1e-9: # Avoid division by zero or very small numbers
        # Check if it's very close to 100% RH, then try to calculate for slightly less
        if abs(p_w - get_saturation_vapor_pressure_kPa(temp_c)) < 1e-6 and rh_percent > 99.99:
             p_w_adjusted = get_saturation_vapor_pressure_kPa(temp_c) * 0.99999
             denominator_adjusted = p_atm_kPa - p_w_adjusted
             if denominator_adjusted <= 1e-9: return float('inf') # Still problematic
             return 0.621945 * p_w_adjusted / denominator_adjusted
        else: # Otherwise, it's likely an invalid state or extreme condition
            return float('inf')
    return 0.621945 * p_w / denominator

def get_enthalpy_kJ_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    w = get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa)
    if w == float('inf'): return float('inf')
    return C_PA * temp_c + w * (H_FG0 + C_PW * temp_c)

def get_temp_from_enthalpy_humidity_ratio(h_kJ_kg, w_kg_kg):
    denominator = C_PA + w_kg_kg * C_PW
    if abs(denominator) < 1e-9: return float('nan')
    return (h_kJ_kg - w_kg_kg * H_FG0) / denominator

# 3. Plotly용 습공기선도 배경 그리기 함수
def add_psychrometric_background_to_fig(fig, temp_range_c, w_range_kg_kg):
    temps_c_plot = np.linspace(temp_range_c[0], temp_range_c[1], 100)

    # Saturation line (RH 100%)
    w_saturated = np.array([get_humidity_ratio_kg_kg(t, 100) for t in temps_c_plot])
    valid_sat_indices = ~np.isinf(w_saturated) & (w_saturated <= w_range_kg_kg[1])
    fig.add_trace(go.Scatter(
        x=temps_c_plot[valid_sat_indices], y=w_saturated[valid_sat_indices],
        mode='lines', name='RH 100%', line=dict(color='black', width=1.5),
        hoverinfo='skip'
    ))

    # Other RH curves
    rh_curves_to_plot = [20, 40, 60, 80]
    for rh in rh_curves_to_plot:
        w_rh = np.array([get_humidity_ratio_kg_kg(t, rh) for t in temps_c_plot])
        valid_rh_indices = ~np.isinf(w_rh) & (w_rh <= w_range_kg_kg[1]) & (w_rh >= w_range_kg_kg[0])
        if np.any(valid_rh_indices):
            fig.add_trace(go.Scatter(
                x=temps_c_plot[valid_rh_indices], y=w_rh[valid_rh_indices],
                mode='lines', name=f'RH {rh}%', line=dict(color='blue', width=0.8, dash='dash'),
                hoverinfo='skip'
            ))

    # Enthalpy lines
    h_min_approx = get_enthalpy_kJ_kg(temp_range_c[0], 20 if temp_range_c[0] > 0 else 80)
    h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 100)

    if h_min_approx == float('inf'): h_min_approx = 0
    if h_max_approx == float('inf'): h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 80, P_ATM)
    if h_max_approx == float('inf'): h_max_approx = 120 # Fallback

    enthalpy_lines_values = np.unique(np.linspace(round(h_min_approx / 10) * 10, round(h_max_approx / 10) * 10, 10).astype(int))
    enthalpy_lines_values = [h for h in enthalpy_lines_values if h != float('inf') and h != float('-inf')]

    for h_val in enthalpy_lines_values:
        if np.isinf(h_val) or np.isnan(h_val): continue
        w_for_h_line = np.linspace(w_range_kg_kg[0], w_range_kg_kg[1], 50)
        temps_for_h_line = np.array([get_temp_from_enthalpy_humidity_ratio(h_val, w) for w in w_for_h_line])

        valid_h_points_indices = ~np.isnan(temps_for_h_line) & \
                                 (temps_for_h_line >= temp_range_c[0]) & \
                                 (temps_for_h_line <= temp_range_c[1]) & \
                                 (w_for_h_line >= w_range_kg_kg[0]) & \
                                 (w_for_h_line <= w_range_kg_kg[1])

        if np.sum(valid_h_points_indices) > 1:
            temps_plot_h = temps_for_h_line[valid_h_points_indices]
            w_plot_h = w_for_h_line[valid_h_points_indices]
            fig.add_trace(go.Scatter(
                x=temps_plot_h, y=w_plot_h, mode='lines',
                name=f'h={h_val}kJ/kg', line=dict(color='green', width=0.8, dash='dot'),
                hoverinfo='skip'
            ))
            # Add enthalpy label (can be tricky to position perfectly, place at one end)
            if len(temps_plot_h) > 0:
                text_x = temps_plot_h[0]
                text_y = w_plot_h[0]
                if temp_range_c[0] <= text_x <= temp_range_c[1] and \
                   w_range_kg_kg[0] <= text_y <= w_range_kg_kg[1]:
                    fig.add_annotation(
                        x=text_x, y=text_y, text=f"{h_val}",
                        showarrow=False, font=dict(color="green", size=9),
                        xanchor="right", yanchor="bottom", textangle=-30
                    )

# 4. 대화형 차트 생성 함수 (Function to create interactive chart with Plotly)
def create_interactive_psychrometric_chart_plotly(csv_data_string):
    try:
        df = pd.read_csv(io.StringIO(csv_data_string))
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    except Exception as e:
        print(f"Error reading or parsing CSV data: {e}")
        return

    if df.empty:
        print("CSV data is empty.")
        return

    stages_config = [
        {'name': "1. Outdoor Air", 'temp_col': 'OA_Temp', 'rh_col': 'OA_RH', 'color': '#1f77b4'},
        {'name': "2. PH/C", 'temp_col': 'PHC_Temp', 'rh_col': 'PHC_RH', 'color': '#ff7f0e'},
        {'name': "3. PC/C", 'temp_col': 'PCC_Temp', 'rh_col': 'PCC_RH', 'color': '#2ca02c'},
        {'name': "4. Cooling Coil", 'temp_col': 'CC_Temp', 'rh_col': 'CC_RH', 'color': '#d62728'},
        {'name': "5. Re-Heating Coil", 'temp_col': 'RHC_Temp', 'rh_col': 'RHC_RH', 'color': '#9467bd'}
    ]

    # Pre-calculate W and H for all points to determine chart bounds
    all_temps_for_bounds = []
    all_ws_for_bounds = []
    for index, row in df.iterrows():
        for stage in stages_config:
            temp = row[stage['temp_col']]
            rh = row[stage['rh_col']]
            all_temps_for_bounds.append(temp)
            w = get_humidity_ratio_kg_kg(temp, rh)
            if w != float('inf') and w < 0.1: # Cap w for bounds calculation
                all_ws_for_bounds.append(w)

    if not all_temps_for_bounds or not all_ws_for_bounds:
        print("No valid data points to plot after psychrometric calculations for bounds.")
        return

    min_temp_data, max_temp_data = min(all_temps_for_bounds), max(all_temps_for_bounds)
    min_w_data, max_w_data = min(all_ws_for_bounds), max(all_ws_for_bounds)

    temp_margin = (max_temp_data - min_temp_data) * 0.1 if (max_temp_data - min_temp_data) > 0 else 5
    w_margin = (max_w_data - min_w_data) * 0.1 if (max_w_data - min_w_data) > 0 else 0.002

    chart_min_temp = min_temp_data - temp_margin
    chart_max_temp = max_temp_data + temp_margin * 1.5
    chart_min_w = 0
    chart_max_w = max_w_data + w_margin * 2.0

    w_sat_at_max_temp_chart = get_humidity_ratio_kg_kg(chart_max_temp, 100)
    if w_sat_at_max_temp_chart != float('inf'):
        chart_max_w = max(chart_max_w, w_sat_at_max_temp_chart * 1.05)
    chart_max_w = min(chart_max_w, 0.035) # Cap max humidity ratio for chart display

    # Create figure
    fig = go.Figure()

    # Add static psychrometric background
    add_psychrometric_background_to_fig(fig, (chart_min_temp, chart_max_temp), (chart_min_w, chart_max_w))

    # Initial data points for the first timestamp (will be placeholders for traces)
    # We'll add one trace for all points and one for all lines to simplify updates in frames
    fig.add_trace(go.Scatter(
        x=[], y=[], mode='markers', marker=dict(size=10, color='black'), name="Process Points",
        text=[], hovertemplate="<b>%{text}</b><br>Temp: %{x:.1f}°C<br>W: %{y:.4f} kg/kg<br>RH: %{customdata[0]:.0f}%<br>H: %{customdata[1]:.1f} kJ/kg<extra></extra>",
        customdata=[]
    ))
    fig.add_trace(go.Scatter(
        x=[], y=[], mode='lines', line=dict(color='crimson', width=2), name="Process Path",
        hoverinfo='skip'
    ))

    # Create frames for the slider
    frames = []
    for idx, row_data in df.iterrows():
        timestamp_str = row_data['Timestamp'].strftime('%Y-%m-%d %H:%M:%S')
        frame_points_x = []
        frame_points_y = []
        frame_points_text = []
        frame_points_customdata = []

        current_annotations = []

        for i, stage_conf in enumerate(stages_config):
            temp_c = row_data[stage_conf['temp_col']]
            rh_p = row_data[stage_conf['rh_col']]

            w = get_humidity_ratio_kg_kg(temp_c, rh_p)
            h = get_enthalpy_kJ_kg(temp_c, rh_p)

            if w != float('inf') and h != float('inf') and w < chart_max_w * 1.5 : # Check if point is in reasonable range
                frame_points_x.append(temp_c)
                frame_points_y.append(w)
                frame_points_text.append(stage_conf['name'])
                frame_points_customdata.append([rh_p, h])

                current_annotations.append(dict(
                    x=temp_c, y=w,
                    text=f"<b>{stage_conf['name']}</b><br>{temp_c}°C, {rh_p}%<br>h={h:.1f}",
                    showarrow=True, arrowhead=0, ax=10, ay=-25, # Simple arrow
                    font=dict(size=9, color=stage_conf['color']),
                    bgcolor="rgba(255,255,255,0.7)"
                ))
            else: # Add placeholder if data is bad, to maintain structure
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_points_text.append(stage_conf['name'] + " (Error)")
                frame_points_customdata.append([None, None])


        frames.append(go.Frame(
            name=timestamp_str,
            data=[
                go.Scatter(x=frame_points_x, y=frame_points_y, text=frame_points_text, customdata=np.array(frame_points_customdata).T if frame_points_customdata else []), # Update points trace
                go.Scatter(x=frame_points_x, y=frame_points_y)  # Update lines trace
            ],
            layout=go.Layout(
                annotations=current_annotations,
                title_text=f"Interactive Psychrometric Chart - {timestamp_str}" # Update title per frame
            )
        ))

    fig.frames = frames

    # Create slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Time: ", "font": {"size": 14}},
        pad={"t": 50, "b":10},
        steps=[]
    )]

    for i, timestamp in enumerate(df['Timestamp']):
        ts_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        slider_step = dict(
            label=timestamp.strftime('%H:%M'), # Short label for step
            method="animate",
            args=[[ts_str], dict(mode="immediate",
                                 frame=dict(duration=100, redraw=True),
                                 transition=dict(duration=50))]
        )
        sliders[0]['steps'].append(slider_step)

    # Initial layout
    initial_timestamp = df['Timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M:%S')
    fig.update_layout(
        title=f"Interactive Psychrometric Chart - {initial_timestamp}",
        xaxis=dict(title='Dry Bulb Temperature (°C)', range=[chart_min_temp, chart_max_temp]),
        yaxis=dict(title='Humidity Ratio (kg_water / kg_dry_air)', range=[chart_min_w, chart_max_w]),
        sliders=sliders,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(l=50, r=50, b=100, t=100), # Adjust margins for title and slider
        hovermode="closest"
    )

    # Set initial data for the first frame explicitly if not done by slider init
    if frames:
        fig.data[len(fig.data)-2].x = frames[0].data[0].x # Points x
        fig.data[len(fig.data)-2].y = frames[0].data[0].y # Points y
        fig.data[len(fig.data)-2].text = frames[0].data[0].text # Points text
        fig.data[len(fig.data)-2].customdata = frames[0].data[0].customdata # Points customdata
        fig.data[len(fig.data)-1].x = frames[0].data[1].x # Lines x
        fig.data[len(fig.data)-1].y = frames[0].data[1].y # Lines y
        if hasattr(frames[0], 'layout') and hasattr(frames[0].layout, 'annotations'):
             fig.layout.annotations = frames[0].layout.annotations
        else:
             fig.layout.annotations = []


    fig.show()


# --- 더미 CSV 데이터 생성 (Dummy CSV Data Generation) ---
dummy_csv_data = """Timestamp,OA_Temp,OA_RH,PHC_Temp,PHC_RH,PCC_Temp,PCC_RH,CC_Temp,CC_RH,RHC_Temp,RHC_RH
2023-07-15 10:00:00,32.0,70,32.1,69.5,28.0,78,18.0,90,20.0,70
2023-07-15 10:10:00,32.2,71,32.3,70.0,27.5,79,15.0,95,22.0,60
2023-07-15 10:20:00,32.5,70,32.6,69.0,27.0,80,13.0,98,24.0,50
2023-07-15 10:30:00,32.8,72,32.9,71.0,26.5,81,12.5,99,24.5,48
2023-07-15 10:40:00,33.0,68,33.1,67.5,26.0,79,12.0,99,25.0,45
2023-07-15 10:50:00,33.1,67,33.2,66.5,25.8,78,11.8,99,25.2,43
2023-07-15 11:00:00,32.9,69,33.0,68.5,26.2,80,12.2,98.5,24.8,46
"""

# --- 실행 (Execution) ---
if __name__ == '__main__':
    create_interactive_psychrometric_chart_plotly(dummy_csv_data)